# Imports

In [ ]:
import numpy as np
import pandas as pd
import nfl_data_py as nfl

# Importing NFL Data

**TODO**: need to unqiuely label columns b/c some currently overlap with each other. Also should remove useless columns from each DF.

In [ ]:
pfr_years = range(2018, 2023 + 1)

# Loading Basic Data

Loading in our current version of the filter data - this takes forever to grab due to the wikipedia queries.

In [ ]:
df_core = pd.read_pickle("../../FantasyData/data-frames/df_basic_ngs_snaps_1999_2023.pkl")

In [ ]:
df_core.head()

In [ ]:
df_core.columns[:60]

# Loading Passing PFR Data

In [ ]:
df_pfr_pass = nfl.import_seasonal_pfr("pass", pfr_years)

In [ ]:
df_pfr_pass.rename(columns={'pfr_id': "pfr_player_id"}, inplace=True)

In [ ]:
df_pfr_pass.head()

In [ ]:
df_pfr_pass.columns

In [ ]:
for col in df_pfr_pass.columns:
    if col in df_core.columns:
        print(col)

# Loading Receiving PFR Data

In [ ]:
df_pfr_rec = nfl.import_seasonal_pfr("rec", pfr_years)

In [ ]:
df_pfr_rec.rename(columns={'pfr_id': "pfr_player_id"}, inplace=True)

In [ ]:
df_pfr_rec.head()

In [ ]:
df_pfr_rec.columns

In [ ]:
for col in df_pfr_rec.columns:
    if col in df_core.columns:
        print(col)

# Loading Rushing PFR Data

In [ ]:
df_pfr_rush = nfl.import_seasonal_pfr("rush", pfr_years)

In [ ]:
df_pfr_rush.rename(columns={'pfr_id': "pfr_player_id"}, inplace=True)

In [ ]:
df_pfr_rush.head()

In [ ]:
df_pfr_rush.columns

In [ ]:
for col in df_pfr_rush.columns:
    if col in df_core.columns:
        print(col)

# Fixing PFR ID in df_core

In [ ]:
# Sort df_core by 'player_name' and 'season' to ensure chronological order
df_core = df_core.sort_values(by=['player_name', 'season'])

# Fill missing 'pfr_player_id' within each group and reset index to align correctly
filled_ids = df_core.groupby('player_name')['pfr_player_id'].apply(lambda x: x.ffill().bfill()).reset_index(level=0, drop=True)

# Assign the filled IDs back to the 'pfr_player_id' column in df_core
df_core['pfr_player_id'] = filled_ids

In [ ]:
df_core.query("player_name == 'Jameson Williams'")[["season", "pfr_player_id"]]

In [ ]:
df_core.query("player_name == 'Kenny Pickett'")[["season", "pfr_player_id"]]

In [ ]:
df_core.query("player_name == 'Mike Evans'")[["season", "pfr_player_id"]]

# Merge in DFs

## PFR Pass

In [ ]:
df_pfr_pass.drop(columns="team", inplace=True)

In [ ]:
df_merge_pass = df_core.merge(df_pfr_pass, on=["season", "pfr_player_id"], how="left")

In [ ]:
df_merge_pass.query("player_name == 'Kenny Pickett'")[["season", "player_name", "attempts", "bad_throws"]]

## PFR Rec

In [ ]:
df_pfr_rec.drop(columns=["age"], inplace=True)

In [ ]:
df_merge_rec = df_merge_pass.merge(df_pfr_rec, on=["season", "pfr_player_id"], how="left")

In [ ]:
df_merge_rec.query("player_name == 'Chris Olave'")[["season", "player_name", "age",
                                                    "receiving_yards", "adot", "rat"]]

## PFR Rush

In [ ]:
df_pfr_rush.drop(columns=["age"], inplace=True)

In [ ]:
df_merge_final = df_merge_rec.merge(df_pfr_rush, on=["season", "pfr_player_id"], how="left")

In [ ]:
df_merge_final.query("player_name == 'Bijan Robinson'")[["season", "player_name", "age",
                                                    "rushing_yards", "yac_att"]]